**<h1>Mikomarket</h1>**

<h1>Wstęp</h1>
Twój znajomy Marek Adamczyk, pasjonat polityki i amatorski inwestor, postanowił "zainwestować" na Mikomarket, nowo otwartym rynku prognoz, przewidując wyniki najbliższych wyborów. Zorientował się jednak, że jednym z znaczących wskaźników są tweety znanych polityków — ich treści momentalnie wpływają na kursy zakładów i giełdy opinii. Niestety, ręczne monitorowanie tych tweetów to wyścig z czasem, a jego umiejętności programistyczne ograniczyły się do stworzenia scrapera, który potrafi pobrać tylko treść tweeta, bez wskazania autora.

Rynek jest bezlitosny, a poświęcenie kilku sekund, aby sprawdzić czy dany post jest autorstwa Korwina, czy Żukowskiej, może kosztować fortunę. Dlatego X przyszedł do Ciebie — swojego zaufanego znajomego-informatyka, z propozycją nie do odrzucenia. Prosi o stworzenie modelu, który w mgnieniu oka przypisze tweet do odpowiedniego polityka, a jako wynagrodzenie zaproponował część swoich zarobków. Pomożesz mu?

# <h2>Dostarczone pliki</h2>

- `train.csv` - Dane treningowe: zawiera treści tweetów oraz ich autorów.
- `test.csv` - Dane testowe: analogiczne do `train.csv`, ale bez oznaczonych autorów (wykorzystasz je do predykcji).
- `przykodp.csv` - Przykładowy plik w formacie w jakim ma być `odpowiedzi.csv`.
- `Zadanie.ipynb` - Notebook, który pomoże Ci rozpocząć pracę nad modelem.


# <h2>Twoje zadanie</h2>

### **1. Stworzenie modelu:**

- Wykorzystaj model allegro/herbert-base-cased.
- Zaimplementuj klasyfikator oparty na HerBERT, który przypisze tweet do jednego z pięciu autorów.
- Wybierz odpowiednie hiperparametry, takie jak learning rate, batch size i liczba epok.
- Możesz użyć np. biblioteki `unicodedata` do dekodowania emotikonów w tweetach.

### **2. Trening i ewaluacja:**

- Podziel dane na zbiór treningowy i walidacyjny.
- Wytrenuj model na zbiorze treningowym.
- Zmierz celność na zbiorze walidacyjnym.

### **3. Predykcja na zbiorze testowym:**

- Wykorzystaj wytrenowany model do przypisania autorów tweetów w zbiorze testowym.
- Wyeksportuj predykcje aby były w takim samym formacie jak jest plik test.csv



# <h2>Ograniczenia</h2>

- Czas działania twojego kodu(trening i ewaluacja) na T4 na Google Colab powinien wynosić maksymalnie 10 minut.
- Do dyspozycji masz model typu BERT: allegro/herbert-base-cased oraz tokenizer allegro/herbert-base-cased. Nie wolno korzystać z innych uprzednio wytrenowanych modeli oraz ze zbiorów danych innych niż dostarczony.
- Twój kod może trenować się tylko i wyłącznie na danych z pliku train.csv.
- Wszystkie dopuszczalne biblioteki są dostępne w pliku requirements.txt


<h2>Ocenianie</h2>

Ocena zależy od wartości accuracy w sposób liniowy, przy czym:

- Jeśli `accuracy < 0.2`, to liczba punktów wynosi 0.
- Jeśli `accuracy >= 0.7`, to liczba punktów wynosi 1.
- Dla wartości `accuracy` pomiędzy 0.2 a 0.7, liczba punktów rośnie liniowo.

Wzór na obliczenie punktów (P):

$$ P = \frac{accuracy - 0.2}{0.5} $$

Gdzie:
- `accuracy` to wartość dokładności modelu (od 0 do 1).
- Jeśli `accuracy < 0.2`, to P = 0.
- Jeśli `accuracy >= 0.7`, to P = 1.

<h2>Rozwiązanie</h2>

- W tym zadaniu musisz musisz dołączyć plik Zadanie.ipynb, który po włączeniu utworzy plik odpowiedzi.csv gdzie będą znajdowały sie odpowiedzi, w formacie takim jak przykodp.csv, oraz plik odpowiedzi.csv.



In [5]:
#Jeśli korzystasz z Google Colaba, odkomentuj poniższą linijkę
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import f1_score, accuracy_score

In [6]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 15.7 MB/s eta 0:00:00


In [7]:
MODEL_NAME = 'allegro/herbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
model = AutoModel.from_pretrained("allegro/herbert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
def set_seed(seed: int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(42)


In [9]:
data = pd.read_csv("train.csv")

categories = data['Label'].unique()
category_to_integer_mapping = {category: idx for idx, category in enumerate(categories)}
integer_to_category_mapping = {idx: category for idx, category in enumerate(categories)}

def pandas_column_to_tensor(column):
    integer_encoded = column.map(category_to_integer_mapping)
    return torch.tensor(integer_encoded.to_numpy(), dtype=torch.long)


def tensor_to_pandas_column(tensor):
    integer_values = tensor.tolist()
    categories = [integer_to_category_mapping[val] for val in integer_values]
    return pd.Series(categories)


In [10]:
import unicodedata

def standardize_emojis(str_to_standarize):
  new_str = ""
  for ch in str_to_standarize:
    if len(ascii(ch)) > 3:
      try:
        name = unicodedata.name(ch)
        if "LATIN" in name.split():
          new_str += ch
          continue
        new_str += ":" + name + ": "

      except:
        new_str += ":UNKNOWN CHAR: "

      continue

    new_str += ch

  return new_str

In [37]:
class TwitterDataset(Dataset):
    def __init__(self, messages, labels, tokenizer, max_len):
        self.messages = messages
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.messages)

    def __getitem__(self, index):
        message = str(self.messages[index])
        label = self.labels[index]

        # Tokenize the input
        encoding = self.tokenizer(
            standardize_emojis(message),
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'message':message,
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': label
        }


In [38]:
class TweetClassifier(nn.Module):
    def __init__(self, base_model_name, num_classes):
        super(TweetClassifier, self).__init__()

        self.herbert = AutoModel.from_pretrained(base_model_name)
        self.classifier = nn.Sequential(
            # nn.Linear(768, 256),
            # nn.BatchNorm1d(768),
            nn.Dropout(0.2),
            # nn.ReLU(),
            nn.Linear(768, num_classes)
        )
        self.classifier = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        # with torch.no_grad():
        herbert_output = self.herbert(input_ids=input_ids, attention_mask=attention_mask)

        cls_embedding = herbert_output.last_hidden_state[:, 0, :]#.pooler_output #

        logits = self.classifier(cls_embedding)

        return logits

In [34]:
def train_model(model, data_loader, loss_fn, optimizer, device):
    model.train()
    all_preds = []
    all_labels = []
    loss_sum = 0
    for batch_num, data in enumerate(data_loader):
        optimizer.zero_grad()

        input_ids = data["input_ids"]
        attention_mask = data["attention_mask"]
        labels = data["label"]

        output = model(input_ids.to(device), attention_mask.to(device))
        loss = loss_fn(output, labels.to(device))

        all_preds.extend(torch.argmax(output, dim=1).cpu().numpy())
        all_labels.extend(labels)

        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
    f1 = f1_score(all_labels, all_preds, average='weighted')
    acc = accuracy_score(all_labels, all_preds)

    print(f"Epoch Loss: {loss_sum}")
    print(f"Training: F1 Score: {f1}, Accuracy: {acc}")

    return model

def validate_model(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in data_loader:
            input_ids = data["input_ids"].to(device)
            attention_mask = data["attention_mask"].to(device)

            outputs = model(input_ids, attention_mask)

            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            labels = data["label"].cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels)

    f1 = f1_score(all_labels, all_preds, average='weighted')
    acc = accuracy_score(all_labels, all_preds)
    print(f"Validation: F1 Score: {f1}, Accuracy: {acc}")
    return f1

def save_predictions(model, data_loader, device, output_file="odpowiedzi.csv"):
    model.eval()
    predictions = []
    tweet_contents = []

    with torch.no_grad():
        for data in data_loader:
            input_ids = data["input_ids"].to(device)
            attention_mask = data["attention_mask"].to(device)

            outputs = model(input_ids, attention_mask)

            preds = torch.argmax(outputs, dim=1).cpu()

            tweet_contents.extend(data["message"])
            predictions.append(preds)

    results_df = pd.DataFrame({"Content": tweet_contents, "Label": tensor_to_pandas_column(torch.cat(predictions, 0))})
    results_df.to_csv(output_file, index=False)

    print(f"Predictions saved to {output_file}")


In [35]:
def prepare_data(train_path, test_path, tokenizer, max_len, val_size=0):

    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path, header=None)
    test_data.columns = ["Content"]

    if val_size > 0:
        split_point = int(len(train_data) * val_size)

        train_data_shuffled = train_data.sample(frac=1, random_state=42).reset_index(drop=True)

        new_train_data = train_data_shuffled.iloc[split_point:].reset_index(drop=True)
        val_data = train_data_shuffled.iloc[:split_point].reset_index(drop=True)

        trainset = TwitterDataset(new_train_data["Content"], pandas_column_to_tensor(new_train_data["Label"]), tokenizer, max_len)
        valset = TwitterDataset(val_data["Content"], pandas_column_to_tensor(val_data["Label"]), tokenizer, max_len)

        trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
        valloader = DataLoader(valset, batch_size=16, shuffle=False)
    else:
        trainset = TwitterDataset(train_data["Content"], pandas_column_to_tensor(train_data["Label"]), tokenizer, max_len)
        val_data = None
        valloader = None
        trainloader = DataLoader(trainset, batch_size=16, shuffle=True)

    testset = TwitterDataset(test_data["Content"], torch.tensor([0 for _ in range(len(test_data["Content"]))], dtype=torch.long), tokenizer, max_len)
    testloader = DataLoader(testset, batch_size=16, shuffle=False)

    if val_size > 0:
        return trainloader, valloader, testloader
    else:
        return trainloader, testloader

In [39]:
MODEL_NAME = 'allegro/herbert-base-cased'
MAX_LEN = 128
BATCH_SIZE = 16
NUM_EPOCHS = 5
NUM_CLASSES = 5
LEARNING_RATE = 2e-5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
trainloader, valloader, testloader = prepare_data("train.csv", "test.csv", tokenizer, MAX_LEN, 0.2)
model = TweetClassifier(MODEL_NAME, NUM_CLASSES).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# train_model(model, trainloader, valloader, criterion, optimizer, NUM_EPOCHS, device, debug=True)

for epoch in range(NUM_EPOCHS):
    model = train_model(model, trainloader, criterion, optimizer, device)
    validate_model(model, valloader, device)

Using device: cuda


Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch Loss: 102.81155002117157
Training: F1 Score: 0.5693883723368378, Accuracy: 0.5706666666666667
Validation: F1 Score: 0.7164135841095168, Accuracy: 0.72
Epoch Loss: 52.86940883100033
Training: F1 Score: 0.8085043058565298, Accuracy: 0.8086666666666666
Validation: F1 Score: 0.7464872789563645, Accuracy: 0.744
Epoch Loss: 30.179605022072792
Training: F1 Score: 0.9035687476188717, Accuracy: 0.9033333333333333
Validation: F1 Score: 0.7528434790713284, Accuracy: 0.7546666666666667
Epoch Loss: 16.337182322517037
Training: F1 Score: 0.953363182780861, Accuracy: 0.9533333333333334
Validation: F1 Score: 0.7468225131584084, Accuracy: 0.7493333333333333
Epoch Loss: 8.662055152468383
Training: F1 Score: 0.974011849722926, Accuracy: 0.974
Validation: F1 Score: 0.7861684031655671, Accuracy: 0.784


In [40]:
save_predictions(model, testloader, device)

Predictions saved to odpowiedzi.csv
